In [1]:
import splusdata
import pandas as pd
import numpy as np
import os

from astropy.coordinates import SkyCoord
import astropy.units as u

bands = ['R', 'G', 'I', 'U', 'Z', 'F378', 'F395', 'F410', 'F430', 'F515', 'F660', 'F861']
downloaded_fits = 'downloaded_fits/'

conn = splusdata.connect ( 'juanpablocalderon', 'choco134' ) 

You have access to internal data


In [33]:
for gal in ['2MASX J03351141-3545324']: #['NGC 1386', 'LEDA 13515', 'NGC 1389', 'NGC 1404', 'NGC 1408', 'NGC 1399']:
    coord = SkyCoord.from_name( gal )
    ra = str(coord.ra.deg)
    dec = str(coord.dec.deg)

    size = 1500

    id  = gal.replace(' ', '')
    for band in bands:
        name = f'{id}_{ra}_{dec}_{size}_{band}.fits.fz'
        try:
            cut = conn.get_cut(ra, dec, size, band)
            cut.writeto ( os.path.join(downloaded_fits, name), overwrite = True )
            print ( f'\r[*] Downloaded ' + name, end = '', flush = True )
        except:
            print ( "\n[!] Error al descargar:", name) # Esto probablemente se deba a que no esta esa coordenada en splus


[*] Downloaded 2MASXJ03351141-3545324_53.79756543985_-35.75900145802999_1500_F861.fits.fz

In [34]:
import glob

for image in glob.glob('downloaded_fits/*_G.fits.fz'):
    print ( image, fits.open(image)[1].header['OBJECT'] )


downloaded_fits/NGC1404_54.7163208_-35.5943917_1500_G.fits.fz SPLUS-s28s34
downloaded_fits/LEDA13515_55.097542_-35.275778_1500_G.fits.fz SPLUS-s27s35
downloaded_fits/NGC1399_54.62117917_-35.45074167_1500_G.fits.fz SPLUS-s27s34
downloaded_fits/NGC1386_54.1926583_-35.999275_1500_G.fits.fz SPLUS-s28s34
downloaded_fits/NGC1389_54.29892806274_-35.7460758875_1500_G.fits.fz SPLUS-s28s34
downloaded_fits/NGC1408_54.8392_-35.5196_1500_G.fits.fz SPLUS-s28s34
downloaded_fits/2MASXJ03351141-3545324_53.79756543985_-35.75900145802999_1500_G.fits.fz SPLUS-s28s33


In [ ]:
=> OBJECT  = 'SPLUS-s27s34'       / name of observed object                        

In [45]:
import splusdata
from astropy.io import fits
import os
import getpass
import shutil
import subprocess
import csv
import re
import pandas as pd
from astropy.table import Table
from functools import reduce

In [36]:
lista_campos = ["SPLUS-s28s33"]
lista_filtros = ("F378", "F395", "F410", "F430", "F515", "F660", "F861", "U", "G", "R", "I", "Z")

In [37]:
for campo in lista_campos:
    print("Procesando:", campo)
    os.makedirs(f'Campos/{campo}/Imagenes_originales/', exist_ok=True)
    for filtro in lista_filtros:
        print("Procesando:", campo, filtro)
        hdu = conn.get_field(campo, filtro)
        hdu.writeto(f"Campos/{campo}/Imagenes_originales/{campo}_{filtro}_swp.fits", overwrite=True)
        hdul1 = fits.open(f"Campos/{campo}/Imagenes_originales/{campo}_{filtro}_swp.fits")
        hdr2 = hdul1[1].header
        #print(hdr2) # Aquí se revisa el header
        hdul1_data = fits.getdata(f"Campos/{campo}/Imagenes_originales/{campo}_{filtro}_swp.fits")
        fits.writeto(f"Campos/{campo}/Imagenes_originales/{campo}_{filtro}_swp.fits", data = hdul1_data, header = hdr2, overwrite = True)
        

Procesando: SPLUS-s28s33
Procesando: SPLUS-s28s33 F378
Procesando: SPLUS-s28s33 F395
Procesando: SPLUS-s28s33 F410
Procesando: SPLUS-s28s33 F430
Procesando: SPLUS-s28s33 F515
Procesando: SPLUS-s28s33 F660
Procesando: SPLUS-s28s33 F861
Procesando: SPLUS-s28s33 U
Procesando: SPLUS-s28s33 G
Procesando: SPLUS-s28s33 R
Procesando: SPLUS-s28s33 I
Procesando: SPLUS-s28s33 Z


In [100]:
from reproject.mosaicking import find_optimal_celestial_wcs
from reproject.mosaicking import reproject_and_coadd
from reproject import reproject_interp, reproject_exact

from astropy.io import fits
import numpy as np

In [97]:
zp = pd.read_csv ( 'tablas/iDR4_zero-points.csv' )

zp.rename ( columns = {'ZP_J0378':'ZP_f378', 'ZP_J0395':'ZP_f395', 'ZP_J0410':'ZP_f410', 
                       'ZP_J0430':'ZP_f430', 'ZP_J0515':'ZP_f515', 'ZP_J0660':'ZP_f660', 'ZP_J0861':'ZP_f861'}, inplace = True )

for f in [ 'SPLUS-s27s34', 'SPLUS-s27s35', 'SPLUS-s28s33', 'SPLUS-s28s34' ]:
    print ( f, 10** ((-0.4) * zp[ zp['Field'] == f ]['ZP_g'].values[0]),  zp[ zp['Field'] == f ]['ZP_g'].values[0] )

SPLUS-s27s34 3.664375746478333e-10 23.59
SPLUS-s27s35 3.661002285465423e-10 23.591
SPLUS-s28s33 3.691475581654127e-10 23.582
SPLUS-s28s34 3.577667699361103e-10 23.616


# Creo imagenes con extension _zp que tienen aplicado el punto de zero

In [102]:
home = '/home/jpcalderon/'
fitsdir = home + '2023/Fornax/notebooks/Campos/'
output_dir = home + '2023/Fornax/notebooks/Campos/'


for filter in bands:
    for field in [ 'SPLUS-s27s34', 'SPLUS-s27s35', 'SPLUS-s28s33', 'SPLUS-s28s34' ]:
        img = fitsdir + field + '/Imagenes_originales/' + field + '_' + filter + '_swp.fits'
        with fits.open ( img ) as hdul:
            data = hdul[0].data
            header = hdul[0].header

        new_img = fitsdir + field + '/Imagenes_originales/' + field + '_' + filter + '_swp_zp.fits'
        try:
            os.remove ( new_img )
        except OSError:
            pass
        new_data = data + (10**((-0.4) * zp[ zp['Field'] == field ]['ZP_' + filter.lower()].values[0]))
        new_header = header
        print ( "[*] New file: ", new_img )
        fits.writeto ( new_img, data = new_data, header = new_header, overwrite = True )
print ( "[*] Done!" )

[*] New file:  /home/jpcalderon/2023/Fornax/notebooks/Campos/SPLUS-s27s34/Imagenes_originales/SPLUS-s27s34_R_swp_zp.fits
[*] New file:  /home/jpcalderon/2023/Fornax/notebooks/Campos/SPLUS-s27s35/Imagenes_originales/SPLUS-s27s35_R_swp_zp.fits
[*] New file:  /home/jpcalderon/2023/Fornax/notebooks/Campos/SPLUS-s28s33/Imagenes_originales/SPLUS-s28s33_R_swp_zp.fits
[*] New file:  /home/jpcalderon/2023/Fornax/notebooks/Campos/SPLUS-s28s34/Imagenes_originales/SPLUS-s28s34_R_swp_zp.fits
[*] New file:  /home/jpcalderon/2023/Fornax/notebooks/Campos/SPLUS-s27s34/Imagenes_originales/SPLUS-s27s34_G_swp_zp.fits
[*] New file:  /home/jpcalderon/2023/Fornax/notebooks/Campos/SPLUS-s27s35/Imagenes_originales/SPLUS-s27s35_G_swp_zp.fits
[*] New file:  /home/jpcalderon/2023/Fornax/notebooks/Campos/SPLUS-s28s33/Imagenes_originales/SPLUS-s28s33_G_swp_zp.fits
[*] New file:  /home/jpcalderon/2023/Fornax/notebooks/Campos/SPLUS-s28s34/Imagenes_originales/SPLUS-s28s34_G_swp_zp.fits
[*] New file:  /home/jpcalderon/

In [103]:
home = '/home/jpcalderon/'
fitsdir = home + '2023/Fornax/notebooks/Campos/'
output_dir = home + '2023/Fornax/notebooks/Campos/'

for filter in ['G']: #bands:
    print ( "[*] Processing filter: " + filter )
    images = [ fits.open(fitsdir + field + '/Imagenes_originales/' + field + '_' + filter + '_swp_zp.fits')[0] for field in [ 'SPLUS-s27s34', 'SPLUS-s27s35' ] ]

    wcs_out, shape_out = find_optimal_celestial_wcs ( images )
    array, footprint = reproject_and_coadd ( images,
                                       wcs_out, shape_out = shape_out,
                                       reproject_function = reproject_interp ) # reproject_exact )
    file = fitsdir + 'NGC1399_' + filter + '-test.fits'
    print ( "[*] Writing file: " + file )
    fits.writeto ( file, array, wcs_out.to_header(), overwrite = True )
    print ( "[*] Done!\n" )

[*] Processing filter: G


[*] Writing file: /home/jpcalderon/2023/Fornax/notebooks/Campos/NGC1399_G-test.fits
[*] Done!



In [ ]:
home = '/home/jpcalderon/'
fitsdir = home + '2023/Fornax/notebooks/Campos/'
output_dir = home + '2023/Fornax/notebooks/Campos/'

for filter in bands:
    print ( "[*] Processing filter: " + filter )
    images = [ fits.open(fitsdir + field + '/Imagenes_originales/' + field + '_' + filter + '_swp.fits')[0] for field in [ 'SPLUS-s27s34', 'SPLUS-s27s35', 'SPLUS-s28s33', 'SPLUS-s28s34' ] ]

    wcs_out, shape_out = find_optimal_celestial_wcs ( images )
    array, footprint = reproject_and_coadd ( images,
                                       wcs_out, shape_out = shape_out,
                                       reproject_function = reproject_interp ) # reproject_exact )
    file = fitsdir + 'NGC1399_' + filter + '.fits'
    print ( "[*] Writing file: " + file )
    fits.writeto ( file, array, wcs_out.to_header(), overwrite = True )
    print ( "[*] Done!\n" )

[*] Processing filter: R


[*] Writing file: /home/jpcalderon/2023/Fornax/notebooks/Campos/NGC1399_R.fits
[*] Done!

[*] Processing filter: G


In [72]:
data = None
for filtr in ['G', 'Z', 'I', 'R']: 
    f =  fits.open(fitsdir + 'NGC1399_' + filtr + '.fits')
    if data is None:
        data = f[0].data
    else:
        data += f[0].data

    if filtr == 'R':
        f[0].data = data
        f.writeto(fitsdir + 'NGC1399_detection.fits', overwrite=True)

EL procedimiento correcto seria, sumar los 4 filtro de banda ancha, stackear la imagen sumada y correr el isophote sobre esa imagen.

In [ ]:
x, y = 12395.807, 10467.678

In [1]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2

#from __future__ import print_function, \
#    division, \
#    absolute_import

import os
import sys
import math
import glob
import copy
import warnings
import subprocess

import sep
import photutils

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
from matplotlib.patches import Ellipse
from matplotlib.ticker import NullFormatter, MaxNLocator, FormatStrFormatter
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
plt.rc('text', usetex=True)

#from astropy import wcs
from astropy.wcs import WCS
from astropy.io import fits
from astropy.visualization import ZScaleInterval, \
    PercentileInterval, \
    AsymmetricPercentileInterval
from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve

# Increase the pixel stack for large image
sep.set_extract_pixstack(1000000)

In [22]:
# Some useful functions from my code
import kungpao

from kungpao import io
from kungpao.galsbp import galSBP
from kungpao import utils
from kungpao import detection
from kungpao import imtools
from kungpao.display import display_single, IMG_CMAP, SEG_CMAP

ModuleNotFoundError: No module named 'kungpao.galsbp'

In [23]:
# This is my shitty code...work in progress, but at least does not depend on Pyraf now
import kungpao
from kungpao.io import save_to_fits
from kungpao.display import display_single
from kungpao.isophote import galSBP, bmodel, plotting

In [57]:
kungpao

<module 'kungpao' from '/home/jpcalderon/.local/lib/python3.7/site-packages/kungpao-0.1.0-py3.7.egg/kungpao/__init__.py'>

In [3]:
home = '/home/jpcalderon/'
fitsdir = home + '2023/Fornax/notebooks/Campos/'
output_dir = home + '2023/Fornax/notebooks/Campos/'

img_fits = fitsdir + 'NGC1399_R.fits'
msk_fits = ''

g_img =  fits.open(img_fits)[0].data

# Maks
#mask = fits.open(msk_fits)[0].data

print(g_img.shape)



(19785, 24770)


# Make the mask for running isophote

In [10]:
home = '/home/jpcalderon/'
fitsdir = home + '2023/Fornax/notebooks/Campos/'
output_dir = home + '2023/Fornax/notebooks/Campos/'

filename = fitsdir + 'NGC1399_R.fits'
with fits.open ( filename ) as hdul:
    hdul.verify('fix')
    image = hdul[0].data   
    header = hdul[0].header
    wcs = WCS ( filename, naxis = 2 )
    
# For running sep
image = image.byteswap().newbyteorder()

img_w, img_h = image.shape

In [49]:
# build numpy masked arrays. In these images, pixels with no 
# information in then are signaled by the value zero. We use 
# that to build the masks.
import numpy as np
import numpy.ma as ma

data_masked = ma.masked_equal(image, np.zeros(shape=image.shape))
#data_masked = ma.masked_invalid(image)

In [54]:
data_masked.fill_value

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [55]:
msk_fits = fitsdir + 'NGC1399_R_mask.fits'
fits.writeto(msk_fits, data = data_masked.fill_value, header = header, overwrite = True)


In [32]:
# Galaxy center: 
#    could be values measured somewhere else, or just use image center as
#    initial guess
x0_ini, y0_ini = 12395.807, 12395.807

# Axis ratio and Position angle:
#     You could measure them elsewhere and fix them; 
#     Or you can determine them using isophote analysis
#     Initial guesses are b/a=0.9, pa=0.0

ba_ini, pa_ini = 0.9, 17.0

# sma_ini: Initial semi-major axis length to start the measure
#     Normally we pixel a small radii to start, but should contain more than a few pixels.
# sma_min, sma_max: Minimum and maximum radius for isophote analysis
#     Usually sma_min == 0, unless there is saturation or the center is masked out
#     Maximum radius can be the size of the image. `Ellipse` will stop automatically when
#     there is no signal

sma_ini, sma_min, sma_max = 10.0, 0.0, 94.54545454545453

# step: step size for relative flux change along the major axis
#     This is in log scale; smaller step means finer radial binning, but poorer S/N per
#     isophote and could lead to failed run.
#     Small step size is not great idea if you want to determine the b/a or PA profile.
step = 5.252525252525252 #( 52/18 ) / 0.55

# stage: behaviour of Ellipse fitting
#     Ellipse can fix certain parameters at the input values or fit everything
#     1  Fit everything, let the center, shape of each isophote to be free
#     2  Fix the central coordinate of every isophote at the x_cen / y_cen position
#     3  Fix the central coordinate and shape of every isophote at input values
#     4  This is the "force photometry" mode, need an external Ellipse output binary 
#        table as template (use `inEllip` parameter)
# Here we start with stage 1
stage = 1

# integrade_mode: how to estimate the intensity along the isophote
#     options are: mean, median, bi-linear
#     For noisy data with masked regions, median seems to work well
integrade_mode = 'median'

# Along each isophote, `ellipse` can perform sigma clipping to remove outliers 
# Even with masks, this can help reduce the impact of contaminations and small structures
# n_clip: number of sigma-clipping to run
# low_clip: lower threshold. 2 means 2-sigma 
# upp_clip: upper threshold
n_clip, low_clip, upp_clip = 2, 3.0, 3.0

# Pixel scale, zeropoint, exposure time: convert measured intensity into surface brightness
# Just fill the HSC coadd values
pix_scale, zeropoint, exptime = 0.55, 25.0, 1.0

# `galSBP` will try the fitting processes a few times if `ellipse` encounters some issues: e.g. 
# initial radius is not good, step size is too small, or fitting radius range is too large. 
# Each time, it will try to make it "easier"
n_try_max = 2

In [56]:
# If you know the center is not far from the image center, you can use a small `sma_ini` to save time.
sma_ini_cen = 6

ell_i_1, bin_i_1 = galSBP.galSBP(
    filename, mask=msk_fits, galX=x0_ini, galY=y0_ini, galQ=ba_ini, galPA=pa_ini,
    iniSma=sma_ini_cen, minSma=sma_min, maxSma=sma_max,
    pix=pix_scale, zpPhoto=zeropoint, expTime=exptime, 
    stage=stage, ellipStep=step,
    uppClip=upp_clip, lowClip=low_clip, nClip=n_clip, 
    maxTry=n_try_max, intMode=integrade_mode)

os.zgtenv: error scanning `/usr/include/iraf.h'


ERROR (572, "Out of memory")


Exception: Can not find the mask: temp_36R5J0K.fits.pl !

In [68]:
# Some useful functions from my code

from kungpao import io
from kungpao.galsbp import galSBP
from kungpao import utils
from kungpao import detection
from kungpao import imtools
from kungpao.display import display_single, IMG_CMAP, SEG_CMAP

ModuleNotFoundError: No module named 'kungpao.galsbp'

In [63]:
# Convolution kernel I used for detecting objects on the image

kernel3 = np.asarray([[0.092163, 0.221178, 0.296069, 0.221178, 0.092163],
                      [0.221178, 0.530797, 0.710525, 0.530797, 0.221178],
                      [0.296069, 0.710525, 0.951108, 0.710525, 0.296069],
                      [0.221178, 0.530797, 0.710525, 0.530797, 0.221178],
                      [0.092163, 0.221178, 0.296069, 0.221178, 0.092163]])

# Directions to my IRAF executbles
ISO = '/home/jpcalderon/2022/Antlia/profile/src/kungpao/kungpao/iraf/linux/x_isophote.e'
TBL = '/home/jpcalderon/2022/Antlia/profile/src/kungpao/kungpao/iraf/linux/x_ttools.e'

In [66]:
ell_1, bin_1 = galSBP.galSBP(filename, mask=msk_fits,
                             galX=x0_ini, galY=y0_ini,
                             galQ=ba_ini, galPA=pa_ini,
                             iniSma=sma_ini, 
                             minSma=sma_min, maxSma=sma_max,
                             pix=pix_scale, zpPhoto=zeropoint,
                             expTime=exptime, 
                             stage=stage,
                             ellipStep=step,
                             isophote=ISO, 
                             xttools=TBL, 
                             uppClip=upp_clip, lowClip=low_clip, 
                             nClip=n_clip, 
                             maxTry=3,
                             intMode=integrade_mode, 
                             saveOut=True, plMask=True,
                             verbose=True, savePng=False, 
                             updateIntens=False)

TypeError: galSBP() got an unexpected keyword argument 'isophote'